In [1]:
import sys
import xarray as xr
import numpy as np
import os
import yaml
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, ParcelsRandom, Variable

sys.path.append('/home/jvalenti/MOAD/analysis-jose/Source')
from OP_functions import *
from OP_Kernels_e3t import *

In [2]:
config = '/home/jvalenti/MOAD/analysis-jose/OParcels/yaml/unidiste3t.yaml'
local = 0
param = load_config1(config)
#Definitions
start = datetime(param['startdate']['year'], param['startdate']['month'], param['startdate']['day']) #Start date
length = param['param']['length'] # Set Time length [days] 
dt = param['param']['dt'] #toggle between - or + to pick backwards or forwards 
N = param['param']['N'] # number of deploying locations
n = param['param']['n'] # 1000   # number of particles per location
dmin = param['param']['dmin'] #minimum depth
dd = param['param']['dd'] #max depth difference from dmin
name = param['file']['name'] #name output file
dtp = param['param']['dtp'] #how often particle released in hours
odt = param['param']['odt'] #how often data is recorded
rrr = param['param']['r'] #radious of particle deployment
distr = param['param']['distr']
MFc = param['param']['MFc']
# Define paths
paths = path(local)
#Set outfall coordinates (Modify to choose other deploying location)    
#coord=xr.open_dataset(paths['coords'],decode_times=False)
clat = param['param']['lats']
clon = param['param']['lons']
#clon, clat = [float(outf_lon)],[float(outf_lat)] 
duration = timedelta(days=length)

In [3]:
def get_timestamps(start,length):
    timestamps=[]
    duration = timedelta(days=length)
    for day in range(duration.days):
        timestamps.append([start + timedelta(days=day)])
    return np.array(timestamps, dtype='datetime64')

In [4]:
#Set deploy locations
if distr == 'hmg':
    clat,clon = p_unidist(N,N)
    N = len(clat)
elif distr == 'trst':
    clat,clon = transect_deploy(clat,clon,N)
elif distr == 'std':
    N = len(param['param']['lats'])
elif distr == 'pd':
    clat, clon, N = pandas_deploy(N,MFc,int(dtp))
    n = 1
    print(N)

x_offset, y_offset, z = p_deploy(N,n,dmin,dd,rrr)

lon = np.zeros([N,n])
lat = np.zeros([N,n])
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])

In [5]:
#Set start date time and the name of the output file

daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.zarr'
outfile = os.path.join(paths['out'], fn)
####BUILD FIELDS FOR SIMULATION######
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W']
filenames,variables=filename_set(start,length,varlist,local)
dimensions = {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw','time': 'time_counter'}
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

#Find file names and variable names
varlist=['US','VS','WL','Diat','Flag','R','T','S','FS','ssh','Bathy','Kz','totdepth','Vol']
filenames,variables=filename_set(start,length,varlist,local)

#Add Stokes Drift fields
dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'}
us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

#Add Vertical diffusivity coefficient field
dimensions = {'lon': 'glamt', 'lat': 'gphit', 'depth': 'depthw','time': 'time_counter'}
Kz = Field.from_netcdf(filenames['Kz'], variables['Kz'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Kz)

#Add fields located at node T
dimensions = {'lon': 'glamt', 'lat': 'gphit', 'depth': 'deptht','time': 'time_counter'}
Diat = Field.from_netcdf(filenames['Diat'], variables['Diat'], dimensions,allow_time_extrapolation=True)
Flag = Field.from_netcdf(filenames['Flag'], variables['Flag'], dimensions,allow_time_extrapolation=True)
R = Field.from_netcdf(filenames['R'], variables['R'], dimensions,allow_time_extrapolation=True)
S = Field.from_netcdf(filenames['S'], variables['S'], dimensions,allow_time_extrapolation=True)
T = Field.from_netcdf(filenames['T'], variables['T'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Diat)
field_set.add_field(Flag)
field_set.add_field(R)
field_set.add_field(S)
field_set.add_field(T)

#Add Bathymetry 2D field
dimensions = {'lon': 'glamt', 'lat': 'gphit'}
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True)
TD = Field.from_netcdf(filenames['totdepth'], variables['totdepth'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Bth)
field_set.add_field(TD)

#Add Volume 3D field
dimensions = {'lon': 'glamt', 'lat': 'gphit', 'depth': 'depthw'}
Vol = Field.from_netcdf(filenames['Vol'], variables['Vol'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Vol)


#Add SSH and Rivers 2D fields
dimensions = {'lon': 'glamt', 'lat': 'gphit','time': 'time_counter'}
SSH = Field.from_netcdf(filenames['ssh'], variables['ssh'], dimensions,allow_time_extrapolation=True)
field_set.add_field(SSH)
Fraser = Field.from_netcdf(filenames['FS'], variables['FS'], dimensions,allow_time_extrapolation=True,timestamps=get_timestamps(start,length))
field_set.add_field(Fraser)

         It will be opened with no decoding. Filling values might be wrongly parsed.


In [6]:
MPParticle = particle_maker(param)
pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z,time=start+timedelta(hours=odt))
pset2 = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z,time=start+timedelta(hours=odt))

In [7]:
KERNELS =  Advection + pset.Kernel(Stokes_drift)+ pset.Kernel(turb_mix)+ pset.Kernel(Displacement)+ pset.Kernel(Unbeaching)
KERNELS_2 =  Advection_2 + pset2.Kernel(Stokes_drift)+ pset.Kernel(turb_mix_2)+ pset.Kernel(Displacement_2)+ pset.Kernel(Unbeaching)

In [8]:
pset2.execute(KERNELS_2,
            runtime=duration, 
            dt=dt,
            output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=odt)),
            recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvection_2Stokes_driftturb_mix_2Displacement_2Unbeaching ==> /tmp/parcels-2894/lib98d87acc2ff12265ccfc780ddb505fca_0.so
INFO: Output files are stored in /home/jvalenti/MOAD/results/Unidiste3t20190101_1n_20190102_1n.zarr.
100%|██████████| 86400.0/86400.0 [01:26<00:00, 993.72it/s]           


In [9]:
pset.execute(KERNELS,
            runtime=duration, 
            dt=dt,
            output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=odt)),
            recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvectionStokes_driftturb_mixDisplacementUnbeaching ==> /tmp/parcels-2894/lib4854ce06ce8b13043f09c399a7540759_0.so
INFO: Output files are stored in /home/jvalenti/MOAD/results/Unidiste3t20190101_1n_20190102_1n.zarr.
100%|██████████| 86400.0/86400.0 [01:22<00:00, 1051.05it/s]          
